In [9]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from PIL import Image
from torchvision import transforms
import os
torch.set_grad_enabled(False)
model = AutoModel.from_pretrained('internlm/internlm-xcomposer2-4khd-7b', torch_dtype=torch.bfloat16, trust_remote_code=True).cuda().eval()
tokenizer = AutoTokenizer.from_pretrained('internlm/internlm-xcomposer2-4khd-7b', trust_remote_code=True)

You are using a model of type internlmxcomposer2 to instantiate a model of type internlm2. This is not supported for all configurations of models and can yield errors.


Set max length to 16384


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.09it/s]
Some weights of InternLMXComposer2ForCausalLM were not initialized from the model checkpoint at internlm/internlm-xcomposer2-4khd-7b and are newly initialized: ['vit.vision_tower.vision_model.post_layernorm.bias', 'vit.vision_tower.vision_model.post_layernorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:

images_parent_path = "/Users/nghia.vo/university/text_mining/internLLM/archive/assets/assets/"
test_path = "/Users/nghia.vo/university/text_mining/internLLM/test.csv"
predictions_path = "/Users/nghia.vo/university/text_mining/internLLM/predict_internLM.csv"
df = pd.read_csv(test_path)
df['Image'] = images_parent_path + df['Image']
df.head()

,Image,Question,Answer
0,/Users/nghia.vo/university/text_mining/internL...,What is the main food in the picture?,chutney
1,/Users/nghia.vo/university/text_mining/internL...,Is this dish vegetarian or non-vegetarian?,vegan
2,/Users/nghia.vo/university/text_mining/internL...,"Based on the following ingredients, classify t...","sweet,sour,salty,spicy"
3,/Users/nghia.vo/university/text_mining/internL...,Is there any chocolate in this food?,no


# Function

In [11]:
def write_to_csvfile(data, path):
    """
    Append new data to a csv file
    Parameters
    ----------
    data : list
        List of data to be written to the csv file
        
    path : str
        Path to the csv file
    """
    if not os.path.exists(path):
        with open(path, 'w') as f:
            f.write('Image,Question,Answer,Predict\n')
    with open(path, 'a') as f:
        f.write(data + '\n')


In [ ]:
file_exists = os.path.exists(predictions_path)
transform = transforms.Compose([
    transforms.Resize((336, 336)),
    transforms.ToTensor()
])
with open(predictions_path, 'a', newline='', encoding='utf-8') as f:
    for i in range(len(df)):
        row = df.iloc[i]
        image_path = row['Image'] + ".jpg"
        question = '<ImageHere>' + row['Question']
        answer = row['Answer']

        # Load and transform the image
       

        # Get model output
        with torch.cuda.amp.autocast():
            response, his = model.chat(tokenizer, query=question, image=image_path, hd_num=55, history=[], do_sample=False, num_beams=3)
        predicted = response[0]['text']

        # Print results
        print(f"Image: {image_path}")
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"Predicted: {predicted}")
        print()

        # Append to CSV
        row_data = pd.DataFrame([[image_path, question, answer, predicted]], 
                                columns=['Image', 'Question', 'Answer', 'Predict'])
        row_data.to_csv(f, index=False, header=not file_exists)  # Append row, add header only if file doesn't exist

        # Ensure header is not written again
        file_exists = True